In [1]:
# Dependencies
from citipy import citipy
import pandas as pd
import numpy as np
import json
import requests
import timeit

from datetime import datetime
from config import weather_api_key

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
longs = np.random.uniform(low=-180.000, high=180.000, size=2000)

lats_longs = zip(lats, longs)
lats_longs

In [3]:
coordinates = list(lats_longs)

In [4]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
print(cities)

['lengshuitan', 'illoqqortoormiut', 'richards bay', 'hermanus', 'hilo', 'mayo', 'port hedland', 'gejiu', 'dhanwar', 'punta arenas', 'nanpiao', 'hambantota', 'kasongo', 'kupang', 'ushuaia', 'mezen', 'vardo', 'bethel', 'nikolskoye', 'kamenskoye', 'mataura', 'lindi', 'takoradi', 'hobart', 'sao filipe', 'monterrey', 'georgetown', 'puerto ayora', 'dikson', 'lompoc', 'marsaxlokk', 'andalan', 'smithers', 'mount isa', 'pirovskoye', 'garden city', 'kapaa', 'east london', 'rikitea', 'arraial do cabo', 'hovd', 'baruun-urt', 'thompson', 'riyadh', 'tasiilaq', 'lavrentiya', 'samalaeulu', 'virginia beach', 'iqaluit', 'vaitupu', 'thinadhoo', 'upernavik', 'taksimo', 'cairns', 'vaini', 'pangnirtung', 'mutoko', 'cherskiy', 'port elizabeth', 'bondo', 'margate', 'vao', 'opunake', 'la rioja', 'qaanaaq', 'hami', 'general roca', 'ahipara', 'ostrovnoy', 'lebu', 'ballina', 'caravelas', 'pisco', 'faanui', 'sur', 'busselton', 'saint george', 'torbay', 'gayny', 'barrow', 'tuktoyaktuk', 'yellowknife', 'albany', 'mu

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
city_data = []
r_count = 1
s_count = 1

for i, city in enumerate(cities):
    
    if (i % 50 == 0 and i >= 50):
        s_count += 1
        r_count = 1

    city_url = url + "&q=" + city.replace(" ","+")
    
    r_count += 1
    
    try:
        city_weather = requests.get(city_url).json()
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })
# Try-except
    except:
        pass


In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lengshuitan,CN,26.4111,111.5956,73.35,97,100,2.26,overcast clouds
1,Richards Bay,ZA,-28.7830,32.0377,68.58,78,0,10.76,clear sky
2,Hermanus,ZA,-34.4187,19.2345,60.80,79,1,19.60,clear sky
3,Hilo,US,19.7297,-155.0900,83.28,75,100,6.91,overcast clouds
4,Mayo,US,38.8876,-76.5119,80.35,55,0,9.22,clear sky
5,Port Hedland,AU,-20.3167,118.5667,75.25,94,100,8.05,light rain
6,Gejiu,CN,23.3060,103.1635,57.96,95,45,2.62,scattered clouds
7,Dhanwar,IN,24.4167,85.9833,84.90,44,0,5.06,clear sky
8,Punta Arenas,CL,-53.1500,-70.9167,42.91,65,75,19.57,broken clouds
9,Nanpiao,CN,41.0982,120.7479,64.20,57,100,7.16,overcast clouds


In [8]:
output_data_file = "Weather_Database/WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")